In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from statistics import mean, median
import scipy
from sklearn.decomposition import PCA
from sklearn import preprocessing
from gprofiler import GProfiler
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
import operator
import qvalue as qv


#Reactome file containing information on pathways, the genes they contain and pathway name, also including the illumina identifier for the genes.

reactome = pd.read_csv('../data/reactome.csv', sep=',', index_col = 0)

def read_reactome(file_name, gene_name_start = "ENSG0"):
    df = pd.read_csv(file_name, sep='\t', header=None)
    
    if gene_name_start == None:
        sub_df = df
    else:
        subset_vec = df[0].str.startswith(gene_name_start)
        sub_df = df.loc[subset_vec]
    
    genes_df = sub_df.groupby(1)[0].apply(list)
    names_df = sub_df.groupby(1)[3].max()
    
    out_df = pd.concat([genes_df,names_df], axis=1)
    out_df.columns = ['genes', 'pathway_name']
    
    return out_df

low_level = read_reactome('../data/Ensembl2Reactome_All_Levels.txt')

def my_pca(df, n_pc=1, normalize=True):
    df = df.dropna(axis = 0, how = 'all')#redundant, but keeping it just in case
    X = df.values.T
    if normalize:
        X2 = preprocessing.scale(X)
    else:
        X2 = X
    pca = PCA(n_components = n_pc)
    pca.fit(X2)
    my_pca.pca = pca  #needed for components
    Xnew = pca.fit_transform(X2)
    out_df = pd.DataFrame(Xnew.transpose(), index=list(range(1,n_pc+1)), columns=df.columns)
    out_df = out_df.transpose()
    
    return out_df, my_pca.pca.components_, my_pca.pca.explained_variance_ratio_

#Importing metabric dataset, dividing up what is clinical/expression data and changing the type of the expression columns to float
metabric_data = pd.read_csv('../data/metabric.csv')
#clinical_data = metabric_data.iloc[:27, :]
expression_data = metabric_data.iloc[27:,:]

#print(expression_data.columns)
dtypedict = {}
for i in expression_data.columns[1:]:
    dtypedict[i] = 'float32'
expression_data = expression_data.astype(dtypedict)



new_clinical_patient = pd.read_csv('../data/brca_metabric/data_clinical_patient.txt', sep='\t', index_col=0).iloc[4:]
new_clinical_sample = pd.read_csv('../data/brca_metabric/data_clinical_sample.txt', sep='\t', index_col=0).iloc[4:]
new_clinical = pd.concat([new_clinical_patient, new_clinical_sample.reindex(new_clinical_patient.index)], axis=1)
new_clinical['Triple Neg'] = new_clinical.apply(lambda row: True if ((row['ER Status'] == 'Negative') 
                                                                     and (row['PR Status'] == 'Negative') 
                                                                     and (row['HER2 Status'] == 'Negative')) else False, axis = 1)

new_clinical['ER-/PR-/HER2+'] = new_clinical.apply(lambda row: True if ((row['ER Status'] == 'Negative') 
                                                                     and (row['PR Status'] == 'Negative') 
                                                                     and (row['HER2 Status'] == 'Positive')) else False, axis = 1)





genes = expression_data['Unnamed: 0'].values.tolist()

gp = GProfiler(return_dataframe = True)
gp = gp.convert(organism='hsapiens',
          query=genes)

gp = gp.loc[gp['n_converted'] == 1]
gp = gp.loc[gp['name'] != 'None']
gp = gp.set_index('incoming')
gprofiler_names = gp
gprofiler_names

dataset = expression_data.set_index('Unnamed: 0') #gene_patient
pca_per_pathway = pd.DataFrame(index=expression_data.columns)

real_gene_names = pd.read_csv('../data/illumina2symbol.txt', sep="\t", index_col = 0)



genes_components_per_pathway = {} #nested dictionary where the 'outer dictionary' is the pathway names as keys and values are 
                                  #another dictionary with genes as keys and components as values

for pathway in reactome.index:
    genes = reactome.loc[pathway, "illumina"]
    genes = literal_eval(genes)
    genes = list(filter(lambda a: a != 'NaN', genes))
    pathwaydata = dataset.loc[genes]
    if pathwaydata.index.empty == True:
        pass
    else:
        pathwaydata = pathwaydata.dropna(axis = 0, how = 'any') #has to be done so the lists match, this makes the dropna in my_pca function obsolete
        presentgenes = pathwaydata.index.values.tolist()
        if len(presentgenes) <= 1:
            pass
        else:
            res, components, explained_variance = my_pca(pathwaydata)
            pathwayname = reactome.loc[pathway, 'pathway_name']
            pca_per_pathway[pathwayname] = res

            components = components.tolist()[0]
            innerdict = {}
            for i in range(0, len(presentgenes)):
                component = components[i]
                gene = genes[i]
                if gene in real_gene_names.index:
                    real_name = real_gene_names.loc[gene, "symbol"]
                    innerdict[real_name] = component
                elif gene in gprofiler_names.index:
                    real_name = gprofiler_names.loc[gene, 'name']
                    innerdict[real_name] = component
                else:
                    innerdict[gene] = component
            sorted_innerdict = sorted(innerdict.items(), key = operator.itemgetter(1), reverse = True)
            genes_components_per_pathway[pathwayname] = [sorted_innerdict, explained_variance.flat[0]]

pca_per_pathway = pca_per_pathway.iloc[1:]

In [2]:
full_df = pd.concat([pca_per_pathway, new_clinical.reindex(pca_per_pathway.index)], axis=1)


In [35]:
from scipy.stats import ttest_ind, mannwhitneyu
import qvalue as qv

clusterframes = {}
receptor_list = ['ER Status', 'PR Status', 'HER2 Status', 'Triple Neg', 'ER-/PR-/HER2+']

for i in receptor_list:
    grouped_by_receptor = full_df.groupby(i)
    for group in grouped_by_receptor:
        #print(group)
        df_cluster = pd.DataFrame(index=full_df.iloc[:,:-33].columns)
        groupname = group[0]
        print(f'{i} == {groupname}')
        df = group[1].iloc[:,:-33]
        group2_df = full_df[full_df[i] != groupname].iloc[:,:-33]
        pvaluelist = []
        group1_mean_list = []
        group2_mean_list = []
        for pathway in df:
            group = df[pathway]
            #print(group)
            group2 = group2_df[pathway]
            test = mannwhitneyu(group, group2)
            #print(test)
            pvaluelist.append(test[1])
            group_mean = group.mean()
            group1_mean_list.append(group_mean)
            group2_mean = group2.mean()
            group2_mean_list.append(group2_mean) 
        
    
        #df_cluster[f'Cluster {groupname}'] = group1_mean_list
        #df_cluster['Other clusters'] = group2_mean_list
        #df_cluster['Fold Change'] = np.log2(abs(df_cluster[f'Cluster {groupname}'])) - np.log2(abs(df_cluster['Other clusters']))
    
        df_cluster['p-values'] = pvaluelist
        qv.qvalues(df_cluster, 'p-values', f'{i} qvalues')
        df_cluster['p-values'] = -np.log10(df_cluster['p-values'])
        df_cluster[f'{i} qvalues'] = -np.log10(df_cluster[f'{i} qvalues'])
        #print(groupname)
    clusterframes[f'{i}'] = df_cluster
        
        
#clusterframes

ER Status == Negative
ER Status == Positive
PR Status == Negative
PR Status == Positive
HER2 Status == Negative
HER2 Status == Positive
Triple Neg == False
Triple Neg == True
ER-/PR-/HER2+ == False
ER-/PR-/HER2+ == True


In [37]:
clusterframes['ER Status']

,p-values,ER Status qvalues
RUNX1 regulates estrogen receptor mediated transcription,224.472619,221.155811
Activation of AMPK downstream of NMDARs,198.176334,195.160555
Transcriptional Regulation by MECP2,190.702842,187.863154
SLC-mediated transmembrane transport,188.588305,185.879960
DAG and IP3 signaling,188.497798,185.879960
PLC beta mediated events,186.894359,184.355702
G-protein mediated events,186.418710,183.946999
Caspase-mediated cleavage of cytoskeletal proteins,182.790808,180.377089
TFAP2 (AP-2) family regulates transcription of growth factors and their receptors,181.851836,179.489270
Cyclin D associated events in G1,175.338235,173.062819


In [40]:
full_clusterframe = pd.DataFrame(index=clusterframes['ER Status'].index)
for i in clusterframes:
    series = clusterframes[i][f'{i} qvalues']
    full_clusterframe = pd.concat([full_clusterframe, series.reindex(full_clusterframe.index)], axis=1)

In [41]:
full_clusterframe

,ER Status qvalues,PR Status qvalues,HER2 Status qvalues,Triple Neg qvalues,ER-/PR-/HER2+ qvalues
RUNX1 regulates estrogen receptor mediated transcription,221.155811,84.308190,28.402013,143.584542,47.965007
Activation of AMPK downstream of NMDARs,195.160555,147.006933,34.695046,140.755584,33.302640
Transcriptional Regulation by MECP2,187.863154,87.384574,30.443113,134.874878,33.619616
SLC-mediated transmembrane transport,185.879960,80.252014,19.738374,145.657094,23.547715
DAG and IP3 signaling,185.879960,99.793291,25.325545,141.757321,25.860778
PLC beta mediated events,184.355702,99.086586,19.032214,143.410767,23.696661
G-protein mediated events,183.946999,98.508600,18.186084,143.584542,23.062481
Caspase-mediated cleavage of cytoskeletal proteins,180.377089,126.052781,35.521822,127.279985,33.147336
TFAP2 (AP-2) family regulates transcription of growth factors and their receptors,179.489270,62.279443,17.073205,129.712769,29.772001
Cyclin D associated events in G1,173.062819,101.188964,24.016694,129.862138,26.944196


In [43]:
full_clusterframe.to_csv("../exp/receptor_qvalues.csv")